In [1]:
import pandas as pd
import numpy as np
import random



# Analysis pre-running
## Comparing function times (used by GP)
Just to have an idea of the time spent by functions (from a possible function set) inside GP. Notice that we analyzed the functions as they are implemented by `gplearn`.

In [2]:
import timeit
fmt = "{:<17}{:.4f}"
print(fmt.format("Sin time:",
                 timeit.timeit(stmt="[np.sin(x) for x in X]", 
                                      setup="import numpy as np;"
                                      "X = np.random.rand(1,1000)", 
                                      number = 10000)))
print(fmt.format("Cos time:", 
                 timeit.timeit(stmt="[np.cos(x) for x in X]", 
                                      setup="import numpy as np;"
                                      "X = np.random.rand(1,1000)", 
                                      number = 10000)))
print(fmt.format("Log time:", 
                 timeit.timeit(stmt="[_protected_log(x) for x in X]", 
                                      setup="from gplearn.functions import _protected_log;"
                                      "import numpy as np;"
                                      "X = np.random.rand(1,1000)", 
                                      number = 10000)))
print(fmt.format("Sqrt time:", 
                 timeit.timeit(stmt="[_protected_sqrt(x) for x in X]", 
                                      setup="from gplearn.functions import _protected_sqrt;"
                                      "import numpy as np;"
                                      "X = np.random.rand(1,1000)", 
                                      number = 10000)))
print(fmt.format("Prot. div time:", 
                 timeit.timeit(stmt="[_protected_division(x,y) for x,y in zip(X,Y)]", 
                                      setup="from gplearn.functions import _protected_division;"
                                      "import numpy as np;"
                                      "X = np.random.rand(1,1000);"
                                      "Y = np.random.rand(1,1000);", 
                                      number = 10000)))
print(fmt.format("AQ time:", 
                 timeit.timeit(stmt="[np.divide(x, np.sqrt(1+y**2)) for x,y in zip(X,Y)]", 
                                      setup="from gplearn.functions import _protected_division;"
                                      "import numpy as np;"
                                      "X = np.random.rand(1,1000);"
                                      "Y = np.random.rand(1,1000);", 
                                      number = 10000)))
print(fmt.format("Mul time:", 
                 timeit.timeit(stmt="[np.multiply(x,y) for x,y in zip(X,Y)]", 
                                      setup="import numpy as np;"
                                      "X = np.random.rand(1,1000);"
                                      "Y = np.random.rand(1,1000);", 
                                      number = 10000)))
print(fmt.format("Sub time:", 
                 timeit.timeit(stmt="[np.subtract(x,y) for x,y in zip(X,Y)]", 
                                      setup="import numpy as np;"
                                      "X = np.random.rand(1,1000);"
                                      "Y = np.random.rand(1,1000);", 
                                      number = 10000)))
print(fmt.format("Add time:",
                 timeit.timeit(stmt="[np.add(x,y) for x,y in zip(X,Y)]", 
                                      setup="import numpy as np;"
                                      "X = np.random.rand(1,1000);"
                                      "Y = np.random.rand(1,1000);", 
                                      number = 10000)))

Sin time:        0.2111
Cos time:        0.2360
Log time:        0.6884
Sqrt time:       0.0874
Prot. div time:  0.3855
AQ time:         0.2070
Mul time:        0.0485
Sub time:        0.0494
Add time:        0.0468


In [81]:
def synthetic_gen(rnd, function, training_gen, test_gen=None):
    training_set = []
    test_set = []
    if not test_gen:
        test_gen = training_gen
    for i in range(training_gen.n):
        inst = training_gen.generate(rnd)
        training_set.append(inst + [function(*inst)])
    for i in range(test_gen.n):
        inst = test_gen.generate(rnd)
        test_set.append(inst + [function(*inst)])
    return [training_set, test_set]

class U:
    def __init__(self, ini, end, n, dim=1):
        self.ini = ini
        self.end = end
        self.n = n
        self.dim = 1
    
    def generate(self, rnd):
        return [rnd.uniform(self.ini, self.end) for i in range(self.dim)]
    

In [82]:
seed = 1234
rnd = random.Random(seed)

data_gen = {"Meier-3": synthetic_gen(rnd, 
                                     lambda x_1,x_2: (x_1**2*x_2**2)/(x_1+x_2), 
                                     U(-1, 1, 50, 2)),
           }#"Meier-4": meier_4(rnd)


TypeError: <lambda>() missing 1 required positional argument: 'x_2'

In [ ]:
Nonic
Sine
Burks
R1
R2
R3
Sext
Koza-2
Koza-3
Poly-10
Korns-1
Korns-2
Korns-3
Korns-4
Korns-5
Korns-6
Korns-7
Korns-8
Korns-9
Korns-10
Korns-11
Korns-12
Vladislavleva-1 (Vlad1)
Vladislavleva-2 (Vlad2)
Vladislavleva-3 (Vlad3)
Vladislavleva-4 (Uball5D)
Vladislavleva-5 (Vlad5)
Vladislavleva-6 (Vlad6)
Vladislavleva-7 (Vlad7)
Vladislavleva-8 (Vlad8)
Pagie-1
Keijzer-1
Keijzer-2
Keijzer-3
Keijzer-4
Keijzer-5
Keijzer-6
Keijzer-7
Keijzer-8
Keijzer-9
Keijzer-10
Keijzer-11
Keijzer-12
Keijzer-13
Keijzer-14
Keijzer-15
Nguyen-1
Nguyen-2
Nguyen-3
Nguyen-4
Nguyen-5
Nguyen-6
Nguyen-7
Nguyen-8
Nguyen-9
Nguyen-10
Nguyen-12

    